In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Load data
df = pd.read_csv("../data/gld_features.csv", index_col='Date', parse_dates=True)

# Definisikan ulang fitur
features = ['Open', 'High', 'Low', 'Close', 'Volume',
            'MA5', 'MA10', 'MA20', 'RSI', 'MACD', 'Signal', 'Return', 'Volatility']

# Target
X = df[features]
y_class = df['Target_UpDown']
y_reg = df['Close'].shift(-1).dropna()
X_reg = X.iloc[:-1]  # drop baris terakhir
y_reg = y_reg  # sudah disesuaikan


In [3]:
# Model klasifikasi
clf_model = RandomForestClassifier(n_estimators=200, max_depth=6, random_state=42)
clf_model.fit(X, y_class)

# Model regresi
reg_model = RandomForestRegressor(n_estimators=200, max_depth=6, random_state=42)
reg_model.fit(X_reg, y_reg)


RandomForestRegressor(max_depth=6, n_estimators=200, random_state=42)

In [4]:
# Ambil baris terakhir
X_future = df[features].tail(1)

# Prediksi harga Close besok
predicted_close = reg_model.predict(X_future)
print(f"📈 Prediksi Harga Close Besok: ${predicted_close[0]:.2f}")

# Prediksi arah naik/turun
predicted_direction = clf_model.predict(X_future)
label = "BUY (Naik)" if predicted_direction[0] == 1 else "SELL (Turun)"
print(f"📊 Prediksi Arah Harga Besok: {label}")


📈 Prediksi Harga Close Besok: $241.67
📊 Prediksi Arah Harga Besok: BUY (Naik)


In [5]:
# Pastikan kolom sinyal tersedia
df["Signal_UpDown"] = df["Target_UpDown"].map({1: "BUY", 0: "SELL"})

# Parameter simulasi
initial_cash = 10000
cash = initial_cash
position = 0
entry_price = 0
returns = []

# Loop dari hari ke-2 (karena sinyal ada di hari sebelumnya)
for i in range(1, len(df)):
    signal = df.iloc[i-1]["Signal_UpDown"]
    price_today = df.iloc[i]["Close"]

    # Beli jika sinyal BUY dan belum punya posisi
    if signal == "BUY" and position == 0:
        position = cash / price_today
        entry_price = price_today
        cash = 0

    # Jual jika sinyal SELL dan sedang pegang posisi
    elif signal == "SELL" and position > 0:
        cash = position * price_today
        profit = cash - initial_cash
        returns.append(profit)
        position = 0

# Hitung nilai akhir (kalau masih pegang emas)
final_cash = cash if position == 0 else position * df.iloc[-1]["Close"]
total_profit = final_cash - initial_cash

print("💰 Hasil Simulasi Trading BUY/SELL:")
print(f"📌 Modal awal      : ${initial_cash:,.2f}")
print(f"📈 Nilai akhir     : ${final_cash:,.2f}")
print(f"🧾 Total Profit    : ${total_profit:,.2f}")
print(f"📊 Total Transaksi : {len(returns)} kali")


💰 Hasil Simulasi Trading BUY/SELL:
📌 Modal awal      : $10,000.00
📈 Nilai akhir     : $11,258.26
🧾 Total Profit    : $1,258.26
📊 Total Transaksi : 121 kali
